## Семинар 2. Выкачиваем Интернет (продолжение).

Эпиграф: *У меня есть дома интернет. Можно я его скачаю на флешку и с собой принесу?*
## Введение

Мы научились скачивать из Интернета HTML-код страницы с заданным адресом. Это здорово, но на практике обычно приходится выкачивать содержимое не одной страницы, а целого сайта или даже многих сайтов --- тысячи или миллионы страниц. Понятно, что перечислить все интересующие нас адреса вручную, чтобы выкачать их по отдельности, в таком случае не получится. На этом семинаре мы выясним, как можно выкачивать страницы оптом, а также научимся получше их чистить от ненужных вещей.

Основной проблемой, которую нужно решить при выкачивании большого количества страниц, --- как узнать адреса всех этих страниц. Мы рассмотрим два подхода.

*Первый подход* обычно применяется, когда нужно загрузить все страницы какого-нибудь крупного ресурса --- например, газеты или форума. Адреса страниц на таких сайтах нередко устроены довольно просто: начинаются они все одинаково, а заканчиваются разными числами. Если внимательно посмотреть на адреса нескольких произвольных страниц, можно довольно быстро выяснить, так ли это и каков допустимый диапазон номеров страниц. В этом случае закачка всех страниц будет представлять собой простой цикл, в котором будут перебираться все номера страниц из этого диапазона.

*Второй подход* обычно применяется в **краулерах** --- программах, которые обходят какой-то фрагмент интернета, собирая информацию с разных сайтов. Краулерами, например, пользуются поисковые системы, чтобы индексировать содержимое сайтов. Краулер начинает работу с одной или нескольких страниц, адреса которых задаются вручную, а затем переходит по всем ссылкам из этих страниц. Каждый раз, когда краулер загружает очередную страницу, он находит на ней не только нужную ему информацию, но и все ссылки, которые добавляются в очередь. Важно при этом помнить, где краулер уже побывал, чтобы не переходить по нескольку раз на одни и те же страницы. В настоящих краулерах применяют и другие ухищрения, например, чтобы выяснить, по каким ссылкам лучше переходить сначала, но мы этого касаться не будем.


## Напоминание

Ссылки в HTML задаются тэгом `a`, а сам адрес находится в атрибуте `href`. Ссылка обязательно должна начинаться с протокола (`http://`); если это не так, это означает, что ссылка указывает на другую страницу на том же сайте. При поиске ссылок в HTML нужно помнить, что между тэгом (`a`) и атрибутом (`href`) могут находиться другие атрибуты и любое количество пробелов.

## Пример

Чтобы было не так скучно, в этот раз мы будем тренироваться не на русском сайте, а на ирландском. Допустим, мы хотим скачать все статьи с http://dil.ie/ --- словаря древнеирландского языка. Если мы зайдем в Search и что-нибудь поищем, то словарь выдаст нам список статей с нашим запросом. В колонке слева будет заголовочное слово, а под ним --- прямая ссылка на словарную статью.

Видно, что у них общее начало и у каждой страницы есть свой номер. Это значит, что мы можем пользоваться перебором номеров. Для этого нам достаточно узнать диапазон --- номера самой первой и самой последней страницы, в данном случае --- от 1 до 43345. Это мы оставляем в качестве упражнения читателю. В результате должно получиться что-то такое:


In [2]:
import urllib.request

def download_page(pageUrl):
    try:
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
    except:
        print('Error at', pageUrl)
        return
    # do something with the downloaded text

commonUrl = 'http://dil.ie/'
for i in range(1, 10):
    pageUrl = commonUrl + str(i)
    download_page(pageUrl)

В функции, отвечающей за загрузку, мы поместили собственно загрузку HTML в блок `try-except`. Это было сделано потому, что зачастую не всем номерам из допустимого диапазона соответствуют реальные страницы. Если страницы с таким адресом не существует, функция `urlopen` вызовет ошибку, которая благодаря `try-except` не вызовет падения всей программы.

#### Замечание про кодировки

Открывая веб-страницу, мы получаем всего лишь последовательность байт - чтобы увидеть текст, ее еще нужно расшифровать. За это отвечает метод `decode`, которому в качестве аргумента передается необходимая кодировка. Аргументом этой функции по умолчанию является "utf-8", но на многих русскоязычных сайтах, особенно старых, вы можете встретить кодировку "cp1251". 

## Важный комментарий

Когда Ваша программа выкачивает много страниц сразу, она создаёт нагрузку на сервер выкачиваемого сайта --- и эта нагрузка намного больше, чем нагрузка от обычного посещения сайта людьми. Если Вы выкачиваете содержимое крупного сайта с одного компьютера, то ничего страшного в этом, скорее всего, нет. Но если это не какой-то крупный ресурс, который владеет мощными серверами, и тем более если страницы с него скачивают несколько человек одновременно, это может создать реальные проблемы владельцам выкачиваемого ресурса. Поэтому, во-первых, нужно всегда выяснять, не доступно ли всё содержимое нужного Вам ресурса по отдельной ссылке (например, так обстоит дело с Википедией), а во-вторых, ставить между обращениями к серверу искуственный временной интервал хотя бы в пару секунд:


In [3]:
import time

time.sleep(2)

## Как почистить текст

В html странице, конечно, всегда много тэгов, скриптов и комментариев. А нам обычно бывает нужен только текст.
Чтобы вытащить из html только текст, можно воспользоваться регулярными выражениями:  

In [ ]:
html_content = '<html>....</html>'  # тут какой-то html

regTag = re.compile('<.*?>', re.DOTALL)  # это рег. выражение находит все тэги
regScript = re.compile('<script>.*?</script>', re.DOTALL) # все скрипты
regComment = re.compile('<!--.*?-->', re.DOTALL)  # все комментарии


# а дальше заменяем ненужные куски на пустую строку
clean_t = regScript.sub("", t)
clean_t = regComment.sub("", clean_t)
clean_t = regTag.sub("", clean_t)

Нужно не забывать, что для отображения на html-странице символов, которых нет на клавиатуре, применяются специальные последовательности символов, начинающиеся с амперсанда (&) и заканчивающиеся точкой с запятой (;). Чтобы получить текст не с такими последовательностями, а с нормальными символами, используется специальная функция в питоне `unescape`:

In [4]:
# если у вас Python3.4+
import html
test_string = 'Петя &amp; Вася'
print( html.unescape(test_string) )

Петя & Вася


In [10]:
print( html.unescape('Специальные символы: &quot; &laquo; &raquo; &spades; &hearts; &clubs; &diams; и так далее') )

Специальные символы: " « » ♠ ♥ ♣ ♦ и так далее


In [ ]:
# если у вас Python3.3 или более ранняя версия 
import html.parser    
html.parser.HTMLParser().unescape('Петя &amp; Вася')